In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json

# Inicializa o driver
driver = webdriver.Chrome()
driver.get("https://sa.sinnc.app/conims/5/IDSSaude.dll")

# Injetar o script de monitoramento
script = open("capture_events.js", "r").read()
driver.execute_script(script)

print("O script foi injetado. Realize ações no navegador e pressione ESC para finalizar.")


O script foi injetado. Realize ações no navegador e pressione ESC para finalizar.
